Dataset QM9/xTB
===

**Autor:** Mateus de Jesus Mendes

# Informações

### Informações do QM9

| *Feature*  | Propriedade/Tipo de Dados | Descrição |
|----------------------------|---------------------------|-----------------|
| `A` | Constante Rotacional | A maior das três constantes rotacionais principais (A,B,C), em GHz. |
| `B` | Constante Rotacional | A constante rotacional intermediária, em GHz. |
| `C` | Constante Rotacional | A menor das três constantes rotacionais principais, em GHz. |
| `Cv` | Capacidade Calorífica | Capacidade calorífica a volume constante (Cv) calculada em 298.15K, em cal/mol K. |
| `G` | Energia Livre de Gibbs | Energia Livre de Gibbs (G) calculada em 298.15K, em Hartree (Ha). |
| `G_atomization` | Energia Livre de Gibbs de Atomização | Energia Livre de Gibbs necessária para dissociar a molécula em seus átomos constituintes isolados em 298.15K, em Ha. |
| `H` | Entalpia | Entalpia (H) calculada em 298.15K, em Hartree (Ha). |
| `H_atomization` | Entalpia de Atomização | Entalpia necessária para dissociar a molécula em seus átomos constituintes isolados em 298.15K, em Ha. |
| `InChI` | Identificador Químico (String) | A string (sequência de caracteres) do International Chemical Identifier (InChI) da molécula (geometria inicial GDB-17). |
| `InChI_relaxed` | Identificador Químico Relaxado (String) | O InChI da molécula após o relaxamento da geometria (otimização B3LYP). |
| `Mulliken_charges` | Cargas Atômicas Parciais | Cargas parciais de Mulliken para cada átomo na molécula (vetor de até 29 elementos, preenchido com zero para moléculas menores). |
| `SMILES` | Representação Molecular (String) | A string Simplified Molecular-Input Line-Entry System (SMILES) da molécula (geometria inicial GDB-17). |
| `SMILES_relaxed` | Representação Molecular Relaxada (String) | O SMILES da molécula após o relaxamento da geometria (otimização B3LYP). |
| `U` | Energia Interna | Energia Interna (U) calculada em 298.15K, em Hartree (Ha). |
| `U0` | Energia Interna a 0K | Energia Interna (U) calculada em 0K (inclui Energia Vibracional do Ponto Zero), em Ha. |
| `U0_atomization` | Energia Interna de Atomização a 0K | Energia Interna necessária para dissociar a molécula em seus átomos constituintes isolados em 0K, em Ha. |
| `U_atomization` | Energia Interna de Atomização | Energia Interna necessária para dissociar a molécula em seus átomos constituintes isolados em 298.15K, em Ha. |
| `alpha` | Polarizabilidade Isotrópica | O valor médio da polarizabilidade estática, em Bohr³ (a₀³). |
| `charges` | Números Atômicos | O número atômico (Z) para cada átomo na molécula (vetor de até 29 elementos, preenchido com zero para moléculas menores). |
| `frequencies` | Frequências Vibracionais | As frequências vibracionais harmônicas da molécula, em cm⁻¹ (número variável de elementos). |
| `gap` | Gap HOMO-LUMO | A diferença de energia entre ϵLUMO e ϵHOMO, em Hartree (Ha). |
| `homo` | Energia HOMO | Energia do Highest Occupied Molecular Orbital (HOMO), em Hartree (Ha). |
| `index` | Identificador | Um identificador inteiro sequencial (1-baseado) para a molécula. |
| `lumo` | Energia LUMO | Energia do Lowest Unoccupied Molecular Orbital (LUMO), em Hartree (Ha). |
| `mu` | Momento de Dipolo | A norma (magnitude) do momento de dipolo, em Debye (D). |
| `num_atoms` | Número de Átomos | O número total de átomos (incluindo H) na molécula. |
| `positions` | Coordenadas Cartesianas | As coordenadas x,y,z de cada átomo (matriz de até 29x3, preenchida com zero para moléculas menores), em Ångström (Å). |
| `r2` | Extensão Espacial Eletrônica | O valor esperado do quadrado da distância eletrônica (⟨R²⟩), em Bohr² (a₀²). |
| `tag` | Etiqueta de Dataset (String) | Uma string de identificação (geralmente 'gdb9') para facilitar a extração e o rastreamento. |
| `zpve` | ZPVE | A Zero Point Vibrational Energy (Energia Vibracional do Ponto Zero), em Hartree (Ha). |

<div style="height:20px"></div>

> **Observações:** O maior desafio será a comparação de precisão, pois:
> 
> - Nível de Teoria Diferente: O QM9 foi calculado com DFT/B3LYP (alta precisão) e o XTB usa um método semi-empírico (alta velocidade). Haverá diferenças inerentes nas propriedades.
> 
> - Geometria Inicial: O RDKit gera a geometria inicial, que o XTB refina. Embora o XTB encontre um mínimo local, esse mínimo pode não ser idêntico ao mínimo do DFT/B3LYP do QM9.

### Propriedades Calculadas pelo xTB

| Tipo de cálculo                              | Opção xTB           | Propriedades obtidas                                                                   |
| -------------------------------------------- | ------------------- | -------------------------------------------------------------------------------------- |
| **Single-point (energia em geometria fixa)** | `--sp`              | Energia total, orbital energies (HOMO/LUMO), dipolo, cargas parciais, energia de Fermi |
| **Otimização geométrica**                    | `--opt`             | Geometria otimizada, energia total, forças, dipolo, cargas                             |
| **Frequências vibracionais**                 | `--hess`            | Frequências, modos normais, energia livre, entropia                                    |
| **Solvatação**                               | `--alpb solvente`   | Energia de solvatação e propriedades ajustadas ao meio                                 |
| **População e orbitais**                     | `--pop`, `--vfukui` | Cargas, Fukui indices, orbitais de fronteira                                           |
| **Dinâmica molecular**                       | `--md`              | Trajetórias, energias, temperatura, pressões                                           |


# Pré-processamento dos Dados

### Importação das Bibliotecas

In [1]:
# Leitura dos dados
import tensorflow_datasets as tfds

# Manipulação de dados
import pandas as pd

# Matemática
import numpy as np
from scipy.spatial.distance import cdist
from scipy.linalg import eigh

# Química
from rdkit.Chem import GetPeriodicTable
from rdkit import Chem
from rdkit.Chem import AllChem

# Automatização
import subprocess
from pathlib import Path

### Leitura dos Dados

In [2]:
ds = tfds.load('qm9', split='train', shuffle_files=False)
ds

2025-10-23 21:08:00.848741: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-23 21:08:00.849062: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-23 21:08:00.878451: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-23 21:08:01.834106: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation or

<_PrefetchDataset element_spec={'A': TensorSpec(shape=(), dtype=tf.float32, name=None), 'B': TensorSpec(shape=(), dtype=tf.float32, name=None), 'C': TensorSpec(shape=(), dtype=tf.float32, name=None), 'Cv': TensorSpec(shape=(), dtype=tf.float32, name=None), 'G': TensorSpec(shape=(), dtype=tf.float32, name=None), 'G_atomization': TensorSpec(shape=(), dtype=tf.float32, name=None), 'H': TensorSpec(shape=(), dtype=tf.float32, name=None), 'H_atomization': TensorSpec(shape=(), dtype=tf.float32, name=None), 'InChI': TensorSpec(shape=(), dtype=tf.string, name=None), 'InChI_relaxed': TensorSpec(shape=(), dtype=tf.string, name=None), 'Mulliken_charges': TensorSpec(shape=(29,), dtype=tf.float32, name=None), 'SMILES': TensorSpec(shape=(), dtype=tf.string, name=None), 'SMILES_relaxed': TensorSpec(shape=(), dtype=tf.string, name=None), 'U': TensorSpec(shape=(), dtype=tf.float32, name=None), 'U0': TensorSpec(shape=(), dtype=tf.float32, name=None), 'U0_atomization': TensorSpec(shape=(), dtype=tf.float3

### Featurização

In [3]:
try:
    PT = GetPeriodicTable()
    def atomic_number(symbol):
        return PT.GetAtomicNumber(symbol)
except Exception:
    # Mapeamentos dos números atômicos
    ATOMIC_NUM = {'H':1, 'C':6, 'N':7, 'O':8, 'F':9}
    def atomic_number(symbol):
        return ATOMIC_NUM[symbol]

# ---------- 1) Construtor da Matriz de Coulomb ----------
def coulomb_matrix(Z, coords, max_atoms=None, diag_exp=2.4, eps=1e-8):
    """
    Z: 1D array-like de números atômicos, shape (n_atoms,)
    coords: array-like shape (n_atoms, 3)
    max_atoms: int, tamanho fixo (pad com zeros se n_atoms < max_atoms)
    diag_exp: expoente para diagonal (0.5 * Z_i**diag_exp)
    eps: pequena constante para evitar divisão por zero (distância 0)
    Returns: Coulomb matrix padded to (max_atoms, max_atoms)
    """
    Z = np.array(Z, dtype=float)
    coords = np.array(coords, dtype=float)
    n = len(Z)
    if max_atoms is None:
        max_atoms = n
    # Matriz de distâncias (n x n)
    dists = cdist(coords, coords)  # Eficiente em C
    # Inicia a matriz
    C = np.zeros((max_atoms, max_atoms), dtype=float)
    # Preencher bloco (n x n)
    for i in range(n):
        for j in range(n):
            if i == j:
                C[i, i] = 0.5 * (Z[i] ** diag_exp)
            else:
                dij = dists[i, j]
                C[i, j] = (Z[i] * Z[j]) / (dij + eps)
    return C

# ---------- 2) Ordenamento / Invariância ----------
def sort_coulomb_matrix(C):
    """
    Ordena linhas/colunas da matriz C por norma L2 decrescente das linhas.
    Retorna a matriz ordenada.
    """
    # Norma por linha
    row_norms = np.linalg.norm(C, axis=1)
    # Organiza índices (descer)
    order = np.argsort(-row_norms)
    C_sorted = C[order][:, order]
    return C_sorted, order

# ---------- 3) Flattening / Trigângulo ----------
def flatten_upper_triangle(C):
    """Retorna vetor com elementos da metade superior (incluindo diagonal)."""
    idx = np.triu_indices_from(C)
    return C[idx]

def flatten_full(C):
    """Flatten completo (row-major)."""
    return C.flatten()

# ---------- 4) Espectro de autovalores ----------
def coulomb_eigenvalues(C, k=None):
    """
    Retorna autovalores (ordenados decrescentemente) do bloco ativo de C.
    k: número de primeiros autovalores a retornar; se None, retorna todos.
    """
    # C pode ter padding zeros; para estabilidade usamos eigh (symmetric)
    evals = eigh(C, eigvals_only=True)
    # Ordem decrescente por valor absoluto (ou valor real)
    order = np.argsort(-np.abs(evals))
    evals_sorted = evals[order]
    if k is not None:
        # pad se necessário
        res = np.zeros(k, dtype=float)
        n = min(len(evals_sorted), k)
        res[:n] = evals_sorted[:n]
        return res
    else:
        return evals_sorted

# ---------- 5) Pipeline completo para uma molécula ----------
def featurize_one_molecule(atom_symbols, coords, max_atoms=29,
                           use='sorted_flatten', diag_exp=2.4):
    """
    atom_symbols: list of chemical symbols e.g. ['C','H','H','O',...]
    coords: numpy array (n_atoms, 3)
    use: 'sorted_flatten' | 'spectrum' | 'flatten_full' | 'upper_triangle'
    Returns: 1D numpy array feature vector
    """
    Z = [atomic_number(s) for s in atom_symbols]
    C = coulomb_matrix(Z, coords, max_atoms=max_atoms, diag_exp=diag_exp)
    if use == 'sorted_flatten':
        Csorted, _ = sort_coulomb_matrix(C)
        feat = flatten_upper_triangle(Csorted)  # Usa-se triangulação para reduzir dimensionalidade
    elif use == 'flatten_full':
        Csorted, _ = sort_coulomb_matrix(C)
        feat = flatten_full(Csorted)
    elif use == 'upper_triangle':
        feat = flatten_upper_triangle(C)
    elif use == 'spectrum':
        feat = coulomb_eigenvalues(C, k=max_atoms)  # length = max_atoms
    else:
        raise ValueError("use deve ser 'sorted_flatten'|'spectrum'|'flatten_full'|'upper_triangle'")
    # Garante que seja finito
    feat = np.nan_to_num(feat, nan=0.0, posinf=0.0, neginf=0.0)
    return feat

### Matriz de Coulumb

In [4]:
def coulomb_matrix(atoms, positions, max_atoms=29):
    """
    Gera uma matriz de Coulomb padronizada (com preenchimento) para uma molécula.

    Parameters
    ----------
    atoms : array-like
        Lista de números atômicos (ex: [6, 1, 1, 8])
    positions : array-like
        Coordenadas 3D correspondentes (shape: [n_atoms, 3])
    max_atoms : int
        Tamanho máximo da molécula no dataset (QM9 tem até 29 átomos)

    Returns
    -------
    M : np.ndarray
        Matriz de Coulomb quadrada de shape (max_atoms, max_atoms)
    """
    n = len(atoms)
    Z = np.array(atoms, dtype=float)
    R = np.array(positions, dtype=float)

    # Cria matriz de distâncias |R_i - R_j|
    dist = np.linalg.norm(R[:, None, :] - R[None, :, :], axis=-1)
    np.fill_diagonal(dist, 1.0)  # Evita divisão por zero

    # Termos fora da diagonal
    M = np.outer(Z, Z) / dist
    # Termos diagonais
    np.fill_diagonal(M, 0.5 * Z ** 2.4)

    # Padroniza para tamanho fixo
    M_padded = np.zeros((max_atoms, max_atoms))
    M_padded[:n, :n] = M

    return M_padded

### Iteração sobre o QM9

In [5]:
# Carrega o dataset (já cacheado)
ds = tfds.load('qm9', split='train', shuffle_files=False)
ds = tfds.as_numpy(ds)  # Converte de tensores do TensorFflow para arrays NumPy

X, y = [], []

for sample in ds:
    atoms = sample['charges'][:sample['num_atoms']]
    positions = sample['positions'][:sample['num_atoms']]
    target = sample['U0']

    M = coulomb_matrix(atoms, positions)
    X.append(M.flatten())  # Flattening para vetor
    y.append(target)

X = np.array(X)
y = np.array(y)
print("Shape de X:", X.shape)
print("Shape de y:", y.shape)

2025-10-23 21:08:02.753259: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:396] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


Shape de X: (130831, 841)
Shape de y: (130831,)


2025-10-23 21:08:25.585397: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Conversão para `DataFrame`

In [6]:
df = pd.DataFrame(X, columns=[f'coulomb_{i}' for i in range(X.shape[1])])
df['target'] = y  # Adiciona a coluna do target

df

,coulomb_0,coulomb_1,coulomb_2,coulomb_3,coulomb_4,coulomb_5,coulomb_6,coulomb_7,coulomb_8,coulomb_9,...,coulomb_832,coulomb_833,coulomb_834,coulomb_835,coulomb_836,coulomb_837,coulomb_838,coulomb_839,coulomb_840,target
0,36.858105,5.494742,5.494749,5.494775,5.494770,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.478931
1,53.358707,6.881704,6.881722,6.881582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-56.525887
2,73.516695,8.315085,8.315085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-76.404701
3,36.858105,30.023042,2.653484,5.649190,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-77.308426
4,36.858105,36.466313,5.625362,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-93.411888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130826,36.858105,23.122967,14.222851,14.319466,11.192810,16.703826,23.870325,14.631114,16.439670,5.488958,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-400.633881
130827,36.858105,28.013272,14.704392,14.704376,14.513087,11.239666,16.593313,23.785131,14.513094,5.494320,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-400.629700
130828,36.858105,27.215062,17.042385,10.577716,10.577701,17.042288,23.177546,11.876705,11.194388,5.491292,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-380.753906
130829,36.858105,27.151837,17.050587,10.556593,9.670858,11.948767,23.277564,17.050675,10.556608,5.487982,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-364.720367


Remoção das colunas de valores nulos:

In [7]:
features = df.columns

for f in features:
    if (df[f] == 0.0).all():
        df.drop(f, axis=1, inplace=True)
df

,coulomb_0,coulomb_1,coulomb_2,coulomb_3,coulomb_4,coulomb_5,coulomb_6,coulomb_7,coulomb_8,coulomb_9,...,coulomb_832,coulomb_833,coulomb_834,coulomb_835,coulomb_836,coulomb_837,coulomb_838,coulomb_839,coulomb_840,target
0,36.858105,5.494742,5.494749,5.494775,5.494770,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-40.478931
1,53.358707,6.881704,6.881722,6.881582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-56.525887
2,73.516695,8.315085,8.315085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-76.404701
3,36.858105,30.023042,2.653484,5.649190,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-77.308426
4,36.858105,36.466313,5.625362,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-93.411888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130826,36.858105,23.122967,14.222851,14.319466,11.192810,16.703826,23.870325,14.631114,16.439670,5.488958,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-400.633881
130827,36.858105,28.013272,14.704392,14.704376,14.513087,11.239666,16.593313,23.785131,14.513094,5.494320,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-400.629700
130828,36.858105,27.215062,17.042385,10.577716,10.577701,17.042288,23.177546,11.876705,11.194388,5.491292,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-380.753906
130829,36.858105,27.151837,17.050587,10.556593,9.670858,11.948767,23.277564,17.050675,10.556608,5.487982,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-364.720367


# Automatização QM9 $\to$ xTB

> 1. Extraindo todos os SMILES do QM9:

In [8]:
smiles_list = []

for i, exemplo in enumerate(ds):
    # Decodifica o SMILES
    smiles = exemplo['SMILES'].decode('utf-8')  # bytes → str
    smiles_list.append(smiles)

2025-10-23 21:08:42.360409: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


> 2. Convertendo SMILES para arquivo `.xyz` com o `rdkit`:

def smiles_to_xyz(smiles, i):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)  # Adiciona hidrogênios
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())  # Gera geometria 3D
    AllChem.UFFOptimizeMolecule(mol)  # Otimiza a geometria inicial
    
    filename = fr'/home/mateus25032/work/Projeto_Final_ML/rdkit_molecules/molecule_{i}.xyz'

    with open(filename, "w") as f:
        f.write(Chem.MolToXYZBlock(mol))

In [9]:
def smiles_to_xyz(smiles, i, output_path="/home/mateus25032/work/Projeto_Final_ML/rdkit_molecules"):
    """
    Gera um arquivo `.xyz` a partir de um SMILES válido, com otimização geométrica via RDKit.

    # Parâmetros
    smiles (str): SMILES da molécula.
    i (int): Índice da molécula no dataset.
    output_path (path, str): Endereço do diretório onde as geometrias serão salvas em arquivos `.xyz`.

    # Retorna
       Retorna `True` se a molécula foi salva com sucesso, `False` caso contrário.
    """
    # Validação do SMILES
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"[ERRO] SMILES inválido: {smiles}")
        return False

    # Adiciona hidrogênios
    mol = Chem.AddHs(mol)

    # Tenta gerar a geometria 3D com ETKDGv3 (método mais moderno e estável)
    params = AllChem.ETKDGv3()
    params.randomSeed = 42
    res = AllChem.EmbedMolecule(mol, params)

    # Se falhar, tenta novamente com coordenadas aleatórias
    if res != 0:
        res = AllChem.EmbedMolecule(mol, useRandomCoords=True)
        if res != 0:
            return False

    # Otimização geométrica com UFF
    try:
        AllChem.UFFOptimizeMolecule(mol)
    except ValueError:
        return False

    # Salva o arquivo .xyz
    filename = f"{output_path}/molecule_{i}.xyz"
    with open(filename, "w") as f:
        f.write(Chem.MolToXYZBlock(mol))

    return True


In [10]:
for i, smiles in enumerate(smiles_list):
    try:
        smiles_to_xyz(smiles, i, output_path="/home/mateus25032/work/Projeto_Final_ML/rdkit_molecules")
    except ValueError:
        continue

> 3. Automatização da linha de comando no `cmd` para o `xTB`:

In [3]:
xtb_path = Path(r'/home/mateus25032/work/Projeto_Final_ML/xtb-linux/bin/xtb') # Diretório do xtb
xyz_dir = Path(r'/home/mateus25032/work/Projeto_Final_ML/rdkit_molecules')  # Diretório com as geometrias (arquivos .xyz)
output_dir = Path('xtb_results')
output_dir.mkdir(exist_ok=True)
gfn_model = 2

def run_xtb(xyz_path, command_line):
    out_path = output_dir / f'{xyz_path.stem}.out'
    with open(out_path, 'w') as f:
        subprocess.run(
            command_line,
            stdout=f,
            stderr=subprocess.STDOUT,
            timeout=1200
        )
    return out_path

Execução para a geometria otimizada:

In [12]:
for xyz_file in xyz_dir.glob('*.xyz'):
    out_file = run_xtb(xyz_file, [str(xtb_path), str(xyz_file), "--opt", f"--gfn", str(gfn_model)])

Execução para o hessiano:

In [ ]:
for xyz_file in xyz_dir.glob('*.xyz'):
    out_file = run_xtb(xyz_file, [str(xtb_path), str(xyz_file), '--hess'])

> 4. *Parsing* e extração das propriedades:

Path de saída dos arquivos `.out` do `xTB`, para serem lidos:

In [ ]:
path = Path(r'/home/mateus25032/work/Projeto_Final_ML/xtb_results')

Definição dos *parsers*:

In [ ]:
def parser(prop, out_path):
    """
    Leitura e extração dos dados de propriedades químicas presentes nos arquivos `.out` gerados pelo xTB.
    
    # Parâmetros
    - prop (str): Propriedade química a ser extraída. Pode assumir os valores: "dipole"

    - out_path (str): Endereço do arquivo .out a ser lido.
    
    # Retorna
    Valor numérico da propriedade química escolhida (float).
    """
    
    match prop:
        case ('dipole'):
            with open(out_path, 'r', encoding='latin-1') as f:
                c = 0
                for line in f:
                    if 'molecular dipole' in line:
                        c += 1
                    if 'full' in line and c == 1:
                        return float(line.split()[-1])
        case 'HOMO':
            with open(out_path, 'r', encoding='latin-1') as f:
                for line in f:
                    if '(HOMO)' in line:
                        return float(line.split()[-2])
        case 'LUMO':
            with open(out_path, 'r', encoding='latin-1') as f:
                for line in f:
                    if '(LUMO)' in line:
                        return float(line.split()[-2])
        case 'ZPE':
            with open(out_path, 'r', encoding='latin-1') as f:
                for line in f:
                    if 'zero point energy' in line:
                        return float(line.split()[-3])
        case 'H':
            with open(out_path, 'r', encoding='latin-1') as f:
                for line in f:
                    if 'TOTAL ENTHALPY' in line:
                        return float(line.split()[-3])
        case 'U0':
            with open(out_path, 'r', encoding='latin-1') as f:
                for line in f:
                    if 'TOTAL ENERGY' in line:
                        return float(line.split()[3])
        case 'G':
            with open(out_path, 'r', encoding='latin-1') as f:
                for line in f:
                    if 'TOTAL FREE ENERGY' in line:
                        return float(line.split()[-3])

Momento de dipolo:

In [ ]:
dipole_list = []

for file in path.glob('*.out'):
    dipole = parser('dipole', file)
    dipole_list.append(dipole)

#dipole_list

Energia do HOMO:

In [ ]:
homo_list = []

for file in path.glob('*.out'):
    E_HOMO = parser('HOMO', file)
    homo_list.append(E_HOMO)

#homo_list

Energia do LUMO:

In [ ]:
lumo_list = []

for file in path.glob('*.out'):
    E_LUMO = parser('LUMO', file)
    lumo_list.append(E_LUMO)

#lumo_list

Energia do gap HOMO - LUMO:

In [ ]:
gap_list = np.array(homo_list) - np.array(lumo_list)

#gap_list

Zero Point Energy:

In [ ]:
zpe_list = []

for file in path.glob('*.out'):
    ZPE = parser('ZPE', file)
    zpe_list.append(ZPE)

#zpe_list

Entalpia:

In [ ]:
enthalpy_list = []

for file in path.glob('*.out'):
    H = parser('H', file)
    enthalpy_list.append(H)

#enthalpy_list

Energia total (`U`):

In [ ]:
R = 8.31446261815324  # J/mol·K
T = 298.15
Eh_to_Jmol = 2625.499638  # 1 Eh = 2625.5 kJ/mol
RT_Eh = (R * T / 1000) / Eh_to_Jmol  # converte R*T para Hartree

U = np.array(enthalpy_list) - RT_Eh

Energia interna total (`U0`):

In [ ]:
U0_xtb_list = []

for file in path.glob('*.out'):
    U0 = parser('U0', file)
    U0_xtb_list.append(U0)

#U0_xtb_list

Energia Livre de Gibbs:

In [ ]:
gibbs_list = []

for file in path.glob('*.out'):
    G = parser('G', file)
    gibbs_list.append(G)

#gibbs_list

> 5. Diferença entre `TOTAL ENERGY` e `U0` $\Longrightarrow$ Futuro $\Delta$-*learning*:

In [ ]:
U0_qm9__list = []

# Extrai U0 do QM9
for i, exemplo in enumerate(ds):
    u0 = exemplo['U0']
    U0_qm9__list.append(u0)

delta_qm9_xtb = np.array(U0_qm9__list[:1]) - np.array(U0_xtb_list)

# Dataset xTB

In [ ]:
xtb_dataset = pd.DataFrame({
    'Dipole': dipole_list,
    'E_HOMO': homo_list,
    'E_LUMO': lumo_list,
    'gap_HOMO-LUMO': gap_list,
    'ZPE': zpe_list,
    'H': enthalpy_list,
    'U': U,
    'U0': U0_xtb_list,
    'G': gibbs_list,
    'Delta': delta_qm9_xtb
})

xtb_dataset

Salva o dataset em formato `.csv`:

In [ ]:
xtb_dataset.to_csv('xtb_dataset.csv')

Referências
===

TENSORFLOW. TensorFlow — Uma plataforma de código aberto para aprendizado de máquina. Disponível em: https://www.tensorflow.org/?hl=pt-br. Acesso em: 24 out. 2025.

TENSORFLOW DATASETS. QM9 — TensorFlow Datasets. Disponível em: https://www.tensorflow.org/datasets/catalog/qm9. Acesso em: 24 out. 2025.

RDKIT. Getting Started in Python — RDKit Documentation. Disponível em: https://www.rdkit.org/docs/GettingStartedInPython.html. Acesso em: 24 out. 2025.

XTB. xTB — Documentation. Disponível em: https://xtb-docs.readthedocs.io/en/latest/. Acesso em: 24 out. 2025.

XTB. xTB — Version 6.7.1 Release Files. Disponível em: https://github.com/grimme-lab/xtb/releases/tag/v6.7.1. Acesso em: 24 out. 2025.